In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav 

# audio.py
import audio

file_name = "../resources/ah_16k.wav"
ah = audio.Audio(*wav.read(file_name))

FileNotFoundError: [Errno 2] No such file or directory: '../resources/ah_16k.wav'

In [ ]:
ah.plot()

## 音声の短時間フーリエ変換

一般的に音声の音素は30msくらい。音素の音響的な特徴は前後の音素にも左右されるので10msぐらいにとると特徴を取得できそう。(切り取ったものをフレームと呼ぶ。その長さをフレーム長と呼ぶ。)

で、それぞれでフーリエ変換を行う。

矩形窓でフレーム長10msとっても両端が不連続な関数だとフーリエ変換は高周波に雑音が残るので、両端が減衰する窓関数を使う。ここではハミング窓を使う。

ハミング窓のフレーム長は20msから80ms程度が一般的に使われる。切り取る際は、10msずらしながら切り取っていく。各フレームは両端部分が近隣のフレームとある程度共有している。

In [ ]:
# フレーム長
ms = list(audio.frame_candidates(ah.rate))[-1]

# ハミング窓
hamming = audio.audio_hamming(ms, ah.rate)

# FFT用の要素数(2のべき乗)
n = len(hamming)

# 周波数軸
freqs = np.fft.fftfreq(n, d=1/ah.rate)[:n//2]

In [ ]:
def stft(a, frame_length, step_length, window):
    """短時間フーリエ変換
    
    step_length(ms)ごとに、オーディオデータ(a)をフレーム長(frame_length(ms))の範囲で切り取っていき、
    それぞれに窓関数(window)を適用し、高速フーリエ変換する。
    """
    
    for frame in a.each_frame(frame_length, step_length):
        windowed = frame * window
        ffted = np.fft.fft(windowed, norm="ortho")[:n//2]
        
        yield np.abs(ffted)

In [ ]:
fted_frames = list(stft(ah, ms, 10, hamming))

# 真ん中のやつ見てみる。
mid = fted_frames[len(fted_frames)//2]
plt.plot(freqs, mid)